In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install "dask[dataframe]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import sys
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nc
import matplotlib.pyplot as plt
#sys.path.insert(0, '/global/homes/x/xzheng/python_lib/')
from netCDF4 import Dataset
import matplotlib as mpl 
import matplotlib.colors as colors
# #os.environ['PROJ_LIB'] = r'/global/cfs/cdirs/e3sm/xzheng/conda/pkgs/proj-7.2.0-h277dcde_2/share/'
# from mpl_toolkits.basemap import Basemap
# import cartopy.crs as ccrs
# from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from sklearn import preprocessing
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import silhouette_samples, silhouette_score

In [4]:
#path2 = ('/content/drive/MyDrive/Data/mock.nc')
#path2 = ('/content/drive/MyDrive/Private/Image_Similarity/mock1.nc')
#path2 = ('/content/drive/MyDrive/Data/mock_v2.nc')
#path2 = ('/content/drive/MyDrive/Data/mock_v2.1.nc')
#path2 = ('/content/drive/MyDrive/Data/mock_v3.nc')
#path2 = ('/content/drive/MyDrive/Data/mock_v3.1.nc')
#path2 = ('/content/drive/MyDrive/Data/mock_v4.nc')
path2 = ('/content/drive/MyDrive/Data/ERA5_meteo_sfc_2021_daily.nc')
#path2 = ('/content/drive/MyDrive/Data/ERA5_meteo_sfc_2021_daily_smalldomain.nc')
#path2 = ('/content/drive/MyDrive/Data/ERA5_meteo_sfc_2021_hourly.nc')
#path2 = ('/content/drive/MyDrive/Data/ERA5_meteo_sfc_2021_hourly_smalldomain.nc')
data = xr.open_dataset(path2, decode_times=False) #To view the date as integers of 0, 1, 2,....
#data = xr.open_dataset(path2)# decode_times=False) #To view the date as integers of 0, 1, 2,....
#data5 = xr.open_dataset(path2) # To view time in datetime format
data

<xarray.Dataset>
Dimensions:    (time: 365, longitude: 41, latitude: 41)
Coordinates:
  * time       (time) float64 0.0 1.0 2.0 3.0 4.0 ... 361.0 362.0 363.0 364.0
  * longitude  (longitude) float32 -33.0 -32.75 -32.5 ... -23.5 -23.25 -23.0
  * latitude   (latitude) float32 43.0 42.75 42.5 42.25 ... 33.5 33.25 33.0
Data variables:
    sp         (time, latitude, longitude) float64 ...
    sst        (time, latitude, longitude) float64 ...
    u10        (time, latitude, longitude) float64 ...
    v10        (time, latitude, longitude) float64 ...
    sshf       (time, latitude, longitude) float64 ...
    slhf       (time, latitude, longitude) float64 ...
    t2m        (time, latitude, longitude) float64 ...
Attributes:
    title:          Daily variables from ERA5 hourly data
    history:        Wed Mar 23 12:01:11 2022: ncrcat ERA5_meteo_sfc_2021_01-0...
    Input dataset:  /global/cscratch1/sd/xzheng/ENA/rawdata/ERA5_meteo_sfc_20...
    NCO:            netCDF Operators version 5.0.1 (Homepage = http://nco.sf....

###Preprocessing###


1.   Apply Physics-based NaN imputation
2.   Transform and Normalize data
3.   PCA reduce data





In [5]:
#Function to input NaN values across variables
def null_fill(input):

  dask_df = input.to_dask_dataframe(dim_order=None, set_index=False)
  pd_df = dask_df.compute()
  pd_df1 = pd_df.iloc[:, 3:]
  df2 = pd_df1[pd_df1.isnull().any(axis=1)]
  lst = list(df2.index.values)
  df2.loc[:] = np.nan
  dt = pd.concat([pd_df1, df2], axis=0)
  dt3 = dt[~dt.index.duplicated(keep='last')]
  dt4 = dt3[['sst', 'sp', 'u10', 'v10', 'sshf', 'slhf', 't2m']]
  pd_df4 = pd_df.iloc[:, 0:5]
  dff = pd_df4[['time', 'longitude', 'latitude', 'sst']]
  df = pd.merge(dff, dt4, left_index=True, right_index=True).drop('sst_y', axis=1)
  df.rename(columns={'sst_x':'sst'}, inplace=True)
  df_rows = pd.DataFrame(df).set_index(["time", "longitude", "latitude"])
  data = xr.Dataset.from_dataframe(df_rows)
  df_rows = pd.DataFrame(df).set_index(["time", "longitude", "latitude"])
  data = xr.Dataset.from_dataframe(df_rows)

  return data

In [6]:
#data = null_fill(data)

In [7]:
def datatransformation(input):
    
    '''This function is used to transform the xarray dataset into a pandas dataframe where the dimension "time" would become the index of the DataFrame and,
      pairs of both dimensions "latitude" and "longitude" will become the columns for each variable'''
    
    # If the given input is a string, the below block will be executed
    if isinstance(input, str) == True:
        data1 = data[input]

        # The below line will convert the xarray into a dask dataframe
        dask_df = data1.to_dask_dataframe(dim_order=None, set_index=False)
        # The below line will convert the dask dataframe into a pandas dataframe
        pd_df = dask_df.compute()

        # The below loop will handle missing values in each and every column of the dataframe by substituting mean of individual columns in the place of the missing values
        for i in pd_df.columns:
          if pd_df[i].isna().sum() > 0:
            pd_df[i].fillna(value=pd_df[i].mean(), inplace=True)
        
        # Since the pandas dataframe (p_df) will also have the dimensions of the xarray as columns, we'll have to remove them. Below code handles that part.
        col = 'time','latitude','longitude'
        fin_df = pd_df.loc[:, ~pd_df.columns.isin(col)]

        # trans_data will be the final dataframe that the function will return.
        trans_data = pd.DataFrame()
        for j in fin_df.columns:
          for i in range(0,pd_df.shape[0]):
              c=(j + '(' + str(pd_df.latitude[i])+','+str(pd_df.longitude[i]) + ')') # Every variable followed by the pairs of latitude and longitude will become the columns
              trans_data.loc[pd_df.time[i], c] = pd_df[j][i] # Based on the column name (var+ (lat,lon)), the correct value of each variable will sit in the right place.

        return trans_data

    # If the input is any thing other than a string, the below code will be executed.
    else:
        # The below line will convert the xarray into a dask dataframe
        dask_df = input.to_dask_dataframe(dim_order=None, set_index=False)
        # The below line will convert the dask dataframe into a pandas dataframe
        pd_df = dask_df.compute()

        # The below loop will handle missing values in each and every column of the dataframe by substituting mean of individual columns in the place of the missing values
        for i in pd_df.columns:
          if pd_df[i].isna().sum() > 0:
            pd_df[i].fillna(value=pd_df[i].mean(), inplace=True)
        
        # Since the pandas dataframe (p_df) will also have the dimensions of the xarray as columns, we'll have to remove them. Below code handles that part.
        col = 'time','latitude','longitude'
        fin_df = pd_df.loc[:, ~pd_df.columns.isin(col)]

        # trans_data will be the final dataframe that the function will return.
        trans_data = pd.DataFrame()
        for j in fin_df.columns:
          for i in range(0,pd_df.shape[0]):
              c=(j + '(' + str(pd_df.latitude[i])+','+str(pd_df.longitude[i]) + ')') # Every variable followed by the pairs of latitude and longitude will become the columns
              trans_data.loc[pd_df.time[i], c] = pd_df[j][i] # Based on the column name (var+ (lat,lon)), the correct value of each variable will sit in the right place.

        return trans_data


def datanormalization(input):
  x = input.values #returns a numpy array
  min_max_scaler = preprocessing.MinMaxScaler()
  x_scaled = min_max_scaler.fit_transform(x)
  trans_data = pd.DataFrame(x_scaled, columns=input.columns, index=input.index)
        
  return trans_data

In [8]:
# trans_data1 = datatransformation(data)
# trans_data = datanormalization(trans1_data)

In [9]:
#trans_data.to_csv("/content/drive/MyDrive/Private/Image_Similarity/daily_trans.csv")

In [10]:
#norm_data = datanormalization(trans_data)

In [11]:
#norm_data.to_csv("/content/drive/MyDrive/Private/Image_Similarity/daily_trans_norm.csv")

In [12]:
# trans_data = pd.read_csv("/content/drive/MyDrive/Private/Image_Similarity/daily_trans.csv")
trans_data = pd.read_csv("/content/drive/MyDrive/Private/Image_Similarity/daily_trans_norm.csv")
trans_data = trans_data.iloc[:,1:]
trans_data.head(3)

,"sst(33.0,-33.0)","sst(33.25,-33.0)","sst(33.5,-33.0)","sst(33.75,-33.0)","sst(34.0,-33.0)","sst(34.25,-33.0)","sst(34.5,-33.0)","sst(34.75,-33.0)","sst(35.0,-33.0)","sst(35.25,-33.0)",...,"t2m(40.75,-23.0)","t2m(41.0,-23.0)","t2m(41.25,-23.0)","t2m(41.5,-23.0)","t2m(41.75,-23.0)","t2m(42.0,-23.0)","t2m(42.25,-23.0)","t2m(42.5,-23.0)","t2m(42.75,-23.0)","t2m(43.0,-23.0)"
0,0.248875,0.251932,0.218729,0.167344,0.118699,0.098418,0.119796,0.130408,0.137461,0.151515,...,0.195031,0.185714,0.178300,0.172120,0.154563,0.143675,0.146293,0.156671,0.173913,0.195476
1,0.258726,0.279664,0.243431,0.190116,0.138874,0.113119,0.128353,0.133286,0.145086,0.164616,...,0.258879,0.255773,0.257252,0.258178,0.246343,0.243416,0.248745,0.258991,0.272482,0.286063
2,0.240647,0.263094,0.258533,0.200309,0.129656,0.091264,0.112380,0.124466,0.145409,0.160901,...,0.305736,0.299503,0.298483,0.299952,0.292361,0.286533,0.292380,0.303016,0.319956,0.337499


In [13]:
classification = [4, 4, 4, 4, 4, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       2, 2, 4, 4, 4, 2, 2, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 6, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 3, 3, 1, 1,
       1, 1, 6, 2, 2, 6, 6, 6, 6, 6, 6, 6, 6]

In [14]:
trans_data['Cluster'] = classification

#Internal Cluster Evaluation

RMSE, Spatial Correlation, Silhouette Coefficient, Calinski Harabaz, Davies Bouldin

In [23]:
def data_centroid(input,formed_clusters): #classification
  Cluster_Centers = {}
  centers = []
  Center = []
  Clusters = n_nor_get_clusters(input,formed_clusters)
  for i in set(formed_clusters):
    Cluster_Centers['Cluster_Center' + str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
    centers.append(Cluster_Centers['Cluster_Center' + str(i)])

  centers_sum = sum(centers)
  centers = []
  n = len(Cluster_Centers['Cluster_Center' + str(i)])
  Centers_sum = np.sum(sum(sum(centers_sum/n)))
  Center.append(Centers_sum)

  return Center

# Function that creates a dictionary that holds the values of dates in each cluster
def get_datewise_clusters(formed_clusters): # classification
  Dates_Cluster = {}
  for i in set(formed_clusters): # classification
    Dates_Cluster['Dates_Cluster'+str(i)] = trans_data.index[trans_data.Cluster == i].to_list()
  return Dates_Cluster


# Function that creats a dictionary that holds all the clusters
def n_nor_get_clusters(input,formed_clusters): # classification
  com_arr = []
  Clusters = {}
  Dates_Cluster = get_datewise_clusters(formed_clusters)
  for i in set(formed_clusters):
    for j in Dates_Cluster['Dates_Cluster'+str(i)]:
      arr = np.array(input.isel(time=j).to_array()) # input is data
      com_arr.append(arr)
    Clusters['Cluster' + str(i)] = np.array(com_arr)
    com_arr = []
  return Clusters

# Function that creates a dictionary that holds all the cluster centers
def n_nor_get_cluster_centers(input,formed_clusters): #classification
  Cluster_Centers = {}
  Clusters = n_nor_get_clusters(input,formed_clusters)
  for i in set(formed_clusters):
    Cluster_Centers['Cluster_Center' + str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
  return Cluster_Centers


# Non-normalized
def handle_missing_values(input):
  var_mean = {}
  for i in input.data_vars:
    if input[i].isnull().sum().item() > 0:
      print(i,'has null values')
      var_mean[str(i) + '_mean'] = input[i].mean().item()
      input[i] = input[i].fillna(var_mean[str(i) + '_mean'])
  return input






def st_rmse(input, formed_clusters):

  '''
  input: 
        datatype: 4-D spatio-temporal xarray

        formed_clusters: 1-D array of cluster labels classifying each data point along the time dimension
                         to a cluster label

  Output:
         
        An N X M matrix whose diagonal is a measure of intra-rmse between data points in a cluster
        while the rest of the values represent the inter-rmse between data points in different clusters.
     
  '''

  # #calling function that transforms our data
  # trans_data = datatransformation(input)

  #   #Normalize data
  # trans_data = datanormalization(trans_data)

  # trans_data['Cluster'] = formed_clusters



  # if null_fill == "yes":

  #   data = null_fill(input)

  #   #calling function that transforms our data
  #   trans_data = datatransformation(data)

  #   #Normalize data
  #   trans_data = datanormalization(trans_data)

  #   trans_data['Cluster'] = formed_clusters

  # else:
  #   #calling function that transforms our data
  #   trans_data = datatransformation(input)

  #   #Normalize data
  #   trans_data = datanormalization(trans_data)

  #   trans_data['Cluster'] = formed_clusters
  

  #Intra RMSE Calculation Function
  def n_nor_intra_rmse(input,formed_clusters):
    sq_diff = []
    intra_rmse = []
    Clusters = n_nor_get_clusters(input,formed_clusters)
    Cluster_Centers = n_nor_get_cluster_centers(input,formed_clusters)

    for i in range(len(Clusters)):
      for j in range(len(Clusters['Cluster' + str(i)])):
        diff = Clusters['Cluster' + str(i)][j] - Cluster_Centers['Cluster_Center' + str(i)]
        Sq_diff = (diff**2)
        sq_diff.append(Sq_diff)
      Sq_diff_sum = sum(sq_diff)
      sq_diff = []
      n = len(Clusters['Cluster' + str(i)])
      Sqrt_diff_sum = np.sqrt(sum(sum(sum(Sq_diff_sum/n))))
      intra_rmse.append(Sqrt_diff_sum)
    return intra_rmse


  # Normalized
  # Function that creates two dictionaries that hold all the clusters and cluster centers
  def nor_get_clusters_and_centers(input,formed_clusters):
    Clusters = {}
    Cluster_Centers = {}
    for i in set(formed_clusters):
      Clusters['Cluster' + str(i)] = np.array(input[input.Cluster == i].drop(columns=['Cluster']))
      Cluster_Centers['Cluster_Center' + str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
    return Clusters,Cluster_Centers


  def nor_intra_rmse(input,formed_clusters):
    intra_rmse = []
    sq_diff = []
    Clusters,Cluster_Centers = nor_get_clusters_and_centers(input,formed_clusters)
    for i in range(len(Clusters)):
      for j in range(len(Clusters['Cluster' + str(i)])):
        diff = Clusters['Cluster' + str(i)][j] - Cluster_Centers['Cluster_Center' + str(i)]
        Sq_diff = (diff**2)
        sq_diff.append(Sq_diff)
      Sq_diff_sum = sum(sum(sq_diff))
      sq_diff = []
      n = len(Clusters['Cluster' + str(i)])
      Sqrt_diff_sum = np.sqrt(Sq_diff_sum/n)
      intra_rmse.append(Sqrt_diff_sum)
    return intra_rmse


########################################       #######################################


  # RMSE Calculation
  def RMSE(input,formed_clusters,normalize=False):
    inter_rmse = []
    avg_cluster = {}

    # if normalize == False:
    input = handle_missing_values(input)
    Clusters = n_nor_get_clusters(input,formed_clusters)
    mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
    for i in range(len(Clusters)):
      avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
    for i in range(len(Clusters)):
      for j in range(len(Clusters)):
        if i == j:
          a = n_nor_intra_rmse(input,formed_clusters)
          mat[i].iloc[j] = round(a[i],2)
        else:
          diff = avg_cluster['avg_cluster' + str(i)] - avg_cluster['avg_cluster' + str(j)]
          Sq_diff = (diff**2)
          #Sq_diff_sum = sum(Sq_diff)
          Sq_diff_sum = sum(sum(sum(Sq_diff)))
          #inter_rmse.append(np.sqrt(Sq_diff_sum))
          n = len(avg_cluster['avg_cluster'+str(i)][0])
          Sqrt_diff_sum = np.sqrt(Sq_diff_sum/n)
          mat[i].iloc[j] = round(Sqrt_diff_sum,2)
          #print('Inter RMSE between cluster',i,'and cluster',j,'is:')
          

    else:
      # trans_data = datatransformation(input)

      # # Data Normalization
      # trans_data = datanormalization(trans_data)

      # # Adding class centers and cluster numbers as columns to the dataframe
      # trans_data['Cluster'] = formed_clusters

      # # Rearranging the columns in the dataframe
      # trans_data = trans_data[['Cluster'] + [c for c in trans_data if c not in ['Cluster']]]
    
      Clusters, Cluster_Centers = nor_get_clusters_and_centers(trans_data,formed_clusters)

      # Doing the below step after finding the cluster centers. Otherwise, we'll be calculating mean on date (index) too.
      #trans_data = trans_data.reset_index()

    
      mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
      for i in range(len(Clusters)):
        avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
      for i in range(len(Clusters)):
        for j in range(len(Clusters)):
          if i == j:
            a = nor_intra_rmse(trans_data,formed_clusters)
            mat[i].iloc[j] = round(a[i],2)
          else:
            diff = avg_cluster['avg_cluster' + str(i)] - avg_cluster['avg_cluster' + str(j)]
            Sq_diff = (diff**2)
            #Sq_diff_sum = sum(Sq_diff)
            Sq_diff_sum = sum(Sq_diff)
            #inter_rmse.append(np.sqrt(Sq_diff_sum))
            Sqrt_diff_sum = np.sqrt(Sq_diff_sum)
            mat[i].iloc[j] = round(Sqrt_diff_sum,2)
            #print('Inter RMSE between cluster',i,'and cluster',j,'is:',inter_rmse.pop())
    return mat
  return RMSE(input, formed_clusters)



def st_spat_corr(input, formed_clusters):

  '''
  input: 
        datatype: 4-D spatio-temporal xarray

        formed_clusters: 1-D array of cluster labels classifying each data point along the time dimension
                         to a cluster label

  Output:
         
        An N X M matrix whose diagonal is a measure of intra-spatial correlation between data points in a cluster
        while the rest of the values represent the inter-spatial correlation between data points in different clusters.
     
  '''

  def pearson_PM(x, y):

    #convert format from netcdf to np array
    #x_form = x.to_numpy()
    #y_form = y.to_numpy()

    #Flatten/transform from 2d to 1d
    X_flat = x.flatten()
    Y_flat = y.flatten()

    #Compute correlation matrix
    corr_mat = np.corrcoef(X_flat, Y_flat)

    #Return entry [0,1]
    return corr_mat[0,1]


  #Intra-spatial correlation coefficient Calculation Function
  import functools 

  def n_nor_intra_sp_corr(input,formed_clusters):
    mylist = []
    intra_sp_corr = []
    Clusters = n_nor_get_clusters(input,formed_clusters)
    Cluster_Centers = n_nor_get_cluster_centers(input,formed_clusters)
    
    for i in range(len(Clusters)):
      mylist = []
      for j in range(len(Clusters['Cluster' + str(i)])):
        corr_coeff = pearson_PM(Clusters['Cluster' + str(i)][j], Cluster_Centers['Cluster_Center' + str(i)])
        #print('i: {}, j: {}, corr_coeff:{}'.format(i, j, corr_coeff))
        mylist.append(corr_coeff)
        average_corr_coeff = sum(mylist) / len(mylist)
      intra_sp_corr.append(average_corr_coeff)
    return intra_sp_corr

  

    #Intra-spatial correlation coefficient Calculation Function
  import functools 

  def sp_corr(input,formed_clusters):
    avg_cluster = {}
    Clusters = n_nor_get_clusters(input,formed_clusters)
    mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
    for i in range(len(Clusters)):
      avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
    for i in range(len(Clusters)):
      for j in range(len(Clusters)):
        if i == j:
          a = n_nor_intra_sp_corr(input,formed_clusters)
          mat[i].iloc[j] = round(a[i],2)
        else:
          corr_coeff = pearson_PM(avg_cluster['avg_cluster' + str(i)], avg_cluster['avg_cluster' + str(j)])
          mat[i].iloc[j] = corr_coeff
            #print('Inter RMSE between cluster',i,'and cluster',j,'is:',inter_rmse.pop())
      return mat

    # Non-normalized
  def handle_missing_values(input):
    var_mean = {}
    for i in input.data_vars:
      if input[i].isnull().sum().item() > 0:
        print(i,'has null values')
        var_mean[str(i) + '_mean'] = input[i].mean().item()
        input[i] = input[i].fillna(var_mean[str(i) + '_mean'])
    return input

  #Normalized - Intra-spatial correlation coefficient Calculation Function
  def nor_intra_sp_corr(input,formed_clusters):
    mylist = []
    intra_sp_corr = []
    Clusters,Cluster_Centers = nor_get_clusters_and_centers(input,formed_clusters)
    
    for i in range(len(Clusters)):
      mylist = []
      for j in range(len(Clusters['Cluster' + str(i)])):
        corr_coeff = pearson_PM(Clusters['Cluster' + str(i)][j], Cluster_Centers['Cluster_Center' + str(i)])
        #print('i: {}, j: {}, corr_coeff:{}'.format(i, j, corr_coeff))
        mylist.append(corr_coeff)
        average_corr_coeff = sum(mylist) / len(mylist)
      intra_sp_corr.append(average_corr_coeff)
    return intra_sp_corr



      # Non-normalized Spatial Correlation Calculation
  def sp_corr(input,formed_clusters,normalize=False):
    inter_sp_corr = []
    avg_cluster = {}

    if normalize == False:
      input = handle_missing_values(input)
      Clusters = n_nor_get_clusters(input,formed_clusters)
      mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
      for i in range(len(Clusters)):
        avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
      for i in range(len(Clusters)):
        for j in range(len(Clusters)):
          if i == j:
            a = n_nor_intra_sp_corr(input,formed_clusters)
            mat[i].iloc[j] = round(a[i],2)
          else:
            corr_coeff = pearson_PM(avg_cluster['avg_cluster' + str(i)], avg_cluster['avg_cluster' + str(j)])
            mat[i].iloc[j] = corr_coeff
            #print('Inter RMSE between cluster',i,'and cluster',j,'is:',inter_rmse.pop())
          

    else:
      # trans_data = datatransformation(input)

      # # Data Normalization
      # trans_data = datanormalization(trans_data)

      # # Adding class centers and cluster numbers as columns to the dataframe
      # trans_data['Cluster'] = formed_clusters

      # # Rearranging the columns in the dataframe
      # trans_data = trans_data[['Cluster'] + [c for c in trans_data if c not in ['Cluster']]]
    
      Clusters, Cluster_Centers = nor_get_clusters_and_centers(trans_data,formed_clusters)

      #Doing the below step after finding the cluster centers. Otherwise, we'll be calculating mean on date (index) too.
      #trans_data = trans_data.reset_index()

    
      mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
      for i in range(len(Clusters)):
        avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
      for i in range(len(Clusters)):
        for j in range(len(Clusters)):
          if i == j:
            a = n_nor_intra_sp_corr(input,formed_clusters)
            mat[i].iloc[j] = round(a[i],2)
          else:
            corr_coeff = pearson_PM(avg_cluster['avg_cluster' + str(i)], avg_cluster['avg_cluster' + str(j)])
            mat[i].iloc[j] = round(corr_coeff, 2)
            #print('Inter RMSE between cluster',i,'and cluster',j,'is:',inter_rmse.pop())
          

    return mat

  return sp_corr(input,formed_clusters)


def st_calinski(input,formed_clusters):

    # Non-normalized 
   #Intra calinski Calculation Function
  def n_nor_intra_calinski(input,formed_clusters):
    sq_diff = []
    intra_calinski = []
    Clusters = n_nor_get_clusters(input,formed_clusters)
    Cluster_Centers = n_nor_get_cluster_centers(input,formed_clusters)

    for i in range(len(Clusters)):
      for j in range(len(Clusters['Cluster' + str(i)])):
        diff = Clusters['Cluster' + str(i)][j] - Cluster_Centers['Cluster_Center' + str(i)]
        Sq_diff = (diff**2)
        sq_diff.append(Sq_diff)
      Sq_diff_sum = sum(sq_diff)
      sq_diff = []
      n = len(Clusters['Cluster' + str(i)])
      Sqrt_diff_sum = np.sqrt(sum(sum(sum(Sq_diff_sum/n))))
      intra_calinski.append(Sqrt_diff_sum)
    return intra_calinski


    # Non-normalized 
    #Intra Calculation Function
  def n_nor_intra_CH(input,formed_clusters):
    sq_diff = []
    intra_CH = []
    Clusters = n_nor_get_clusters(input,formed_clusters)
    Cluster_Centers = n_nor_get_cluster_centers(input,formed_clusters)

    for i in range(len(Clusters)):
      for j in range(len(Clusters['Cluster' + str(i)])):
        diff = Clusters['Cluster' + str(i)][j] - Cluster_Centers['Cluster_Center' + str(i)]
        Sq_diff = (diff**2)
        sq_diff.append(Sq_diff)
      Sq_diff_sum = sum(sq_diff)
      sq_diff = []
      n = len(Clusters['Cluster' + str(i)])
      Sqrt_diff_sum = np.sqrt(sum(sum(sum(Sq_diff_sum/n))))
      intra_CH.append(Sqrt_diff_sum)
    return intra_CH


  # Normalized

  def nor_intra_CH(input,formed_clusters):
    intra_CH = []
    sq_diff = []
    #Clusters = n_nor_get_clusters(input,formed_clusters)
    #Cluster_Centers = n_nor_get_cluster_centers(input,formed_clusters)
    Clusters,Cluster_Centers = n_nor_get_cluster_centers(input,formed_clusters)
    for i in range(len(Clusters)):
      for j in range(len(Clusters['Cluster' + str(i)])):
        diff = Clusters['Cluster' + str(i)][j] - Cluster_Centers['Cluster_Center' + str(i)]
        Sq_diff = (diff**2)
        sq_diff.append(Sq_diff)
      Sq_diff_sum = sum(sum(sq_diff))
      sq_diff = []
      n = len(Clusters['Cluster' + str(i)])
      Sqrt_diff_sum = np.sqrt(Sq_diff_sum/n)
      intra_CH.append(Sqrt_diff_sum)
    return intra_CH


  def nor_inter_CH(input,formed_clusters):
    inter_CH = []
    sq_diffs = []
    #Cluster_Centers = n_nor_get_cluster_centers(input,formed_clusters)
    Clusters,Cluster_Centers = n_nor_get_cluster_centers(input,formed_clusters)
    for i in range(len(Cluster_Centers)):
      diff = Cluster_Centers['Cluster_Center' + str(i)] - data_center                     
      Sq_diff = (diff**2)
      sq_diffs.append(Sq_diff)
    Sq_diff_sum = sum(sq_diffs)
    sq_diffs = []
    n = len(Cluster_Centers['Cluster_Center' + str(i)] )
    Sum_diff = np.sum(sum(sum(Sq_diff_sum/n)))
    inter_CH.append(Sum_diff)
    return inter_CH


  # Calinski_Harabasz Calculation
  def calinski_harabasz(input,formed_clusters,normalize=False):
    inter_CH = []
    avg_cluster = {}

    if normalize == False:
      input = handle_missing_values(input)
      Clusters = n_nor_get_clusters(input,formed_clusters)
      mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
      for i in range(len(Clusters)):
        avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
      for i in range(len(Clusters)):
        for j in range(len(Clusters)):
          if i == j:
            a = n_nor_intra_CH(input,formed_clusters)
            mat[i].iloc[j] = round(a[i],2)
          else:
            b = nor_inter_CH(input,formed_clusters)
            mat[i].iloc[i] = round(b[i],2)
        

    else:
    #   trans_data = datatransformation(input)

    # # Data Normalization
    #   trans_data = datanormalization(trans_data)

    # # Adding class centers and cluster numbers as columns to the dataframe
    # trans_data['Cluster'] = formed_clusters

    # # Rearranging the columns in the dataframe
    #   trans_data = trans_data[['Cluster'] + [c for c in trans_data if c not in ['Cluster']]]
  
      Clusters, Cluster_Centers = nor_get_clusters_and_centers(trans_data,formed_clusters)

    # Doing the below step after finding the cluster centers. Otherwise, we'll be calculating mean on date (index) too.
    #trans_data = trans_data.reset_index()

  
      mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
      for i in range(len(Clusters)):
        avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
      for i in range(len(Clusters)):
        for j in range(len(Clusters)):
          if i == j:
            a = n_nor_intra_CH(input,formed_clusters)
            mat[i].iloc[j] = round(a[i],2)
          else:
            b = nor_inter_CH(input,formed_clusters)
            mat[i].iloc[i] = round(b[i],2)
    return mat

  return calinski_harabasz(input,formed_clusters)



def st_davies(input,formed_clusters, normalize=True):


    # Non-normalized 
  #Intra Davies-Bouldin Calculation Function
  def n_nor_intra_db(input,formed_clusters):
    sq_diff = []
    intra_db = []
    Clusters = n_nor_get_clusters(input,formed_clusters)
    Cluster_Centers = n_nor_get_cluster_centers(input,formed_clusters)

    for i in range(len(Clusters)):
      for j in range(len(Clusters['Cluster' + str(i)])):
        diff = Clusters['Cluster' + str(i)][j] - Cluster_Centers['Cluster_Center' + str(i)]
        Sq_diff = (diff**2)
        sq_diff.append(Sq_diff)
      Sq_diff_sum = sum(sq_diff)
      sq_diff = []
      n = len(Clusters['Cluster' + str(i)])
      Sqrt_diff_sum = np.sqrt(sum(sum(sum(Sq_diff_sum/n))))
      intra_db.append(Sqrt_diff_sum)
    return intra_db



    # Normalized
    # Function that creates two dictionaries that hold all the clusters and cluster centers
  def nor_get_clusters_and_centers(input,formed_clusters):
    Clusters = {}
    Cluster_Centers = {}
    for i in set(formed_clusters):
      Clusters['Cluster' + str(i)] = np.array(input[input.Cluster == i].drop(columns=['Cluster']))
      Cluster_Centers['Cluster_Center' + str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
    return Clusters,Cluster_Centers


    # Normalized

  def nor_intra_db(input,formed_clusters):
    intra_db = []
    sq_diff = []
    Clusters,Cluster_Centers = nor_get_clusters_and_centers(input,formed_clusters)
    for i in range(len(Clusters)):
      for j in range(len(Clusters['Cluster' + str(i)])):
        diff = Clusters['Cluster' + str(i)][j] - Cluster_Centers['Cluster_Center' + str(i)]
        Sq_diff = (diff**2)
        sq_diff.append(Sq_diff)
      Sq_diff_sum = sum(sum(sq_diff))
      sq_diff = []
      n = len(Clusters['Cluster' + str(i)])
      Sqrt_diff_sum = np.sqrt(Sq_diff_sum/n)
      intra_db.append(Sqrt_diff_sum)
    return intra_db



    # Davies-Bouldin Calculation
  def davies_b(input,formed_clusters):
    inter_rmse = []
    avg_cluster = {}

    if normalize == False:
      input = handle_missing_values(input)
      Clusters = n_nor_get_clusters(input,formed_clusters)
      mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
      for i in range(len(Clusters)):
        avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
      for i in range(len(Clusters)):
        for j in range(len(Clusters)):
          if i == j:
            a = n_nor_intra_db(input,formed_clusters)
            mat[i].iloc[j] = round(a[i],2)
          else:
            diff = avg_cluster['avg_cluster' + str(i)] - avg_cluster['avg_cluster' + str(j)]
            Sq_diff = (diff**2)
            #Sq_diff_sum = sum(Sq_diff)
            Sq_diff_sum = sum(sum(sum(Sq_diff)))
            #inter_rmse.append(np.sqrt(Sq_diff_sum))
            n = len(avg_cluster['avg_cluster'+str(i)][0])
            Sqrt_diff_sum = np.sqrt(Sq_diff_sum/n)
            mat[i].iloc[j] = round(Sqrt_diff_sum,2)
          #print('Inter RMSE between cluster',i,'and cluster',j,'is:',inter_rmse.pop())
        

    else:
      # trans_data = datatransformation(data)

    # # Data Normalization
    # trans_data = datanormalization(trans_data)

    # # Adding class centers and cluster numbers as columns to the dataframe
    # trans_data['Cluster'] = formed_clusters

    # # Rearranging the columns in the dataframe
    # trans_data = trans_data[['Cluster'] + [c for c in trans_data if c not in ['Cluster']]]
  
      Clusters, Cluster_Centers = nor_get_clusters_and_centers(trans_data,formed_clusters)

    # Doing the below step after finding the cluster centers. Otherwise, we'll be calculating mean on date (index) too.
    #trans_data = trans_data.reset_index()

  
      mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
      for i in range(len(Clusters)):
        avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
      for i in range(len(Clusters)):
        for j in range(len(Clusters)):
          if i == j:
            a = nor_intra_db(trans_data,formed_clusters)
            mat[i].iloc[j] = round(a[i],2)
          else:
            diff = avg_cluster['avg_cluster' + str(i)] - avg_cluster['avg_cluster' + str(j)]
            Sq_diff = (diff**2)
            #Sq_diff_sum = sum(Sq_diff)
            Sq_diff_sum = sum(Sq_diff)
             #inter_rmse.append(np.sqrt(Sq_diff_sum))
            Sqrt_diff_sum = np.sqrt(Sq_diff_sum)
            mat[i].iloc[j] = round(Sqrt_diff_sum,2)
          #print('Inter RMSE between cluster',i,'and cluster',j,'is:',inter_rmse.pop())

    return mat

  return davies_b(input,formed_clusters)

def silhouette_score1(input, formed_clusters, *, metric="euclidean", sample_size=None, random_state=None, **kwds):
  # X0=datatransformation(input)
  # X1 = datanormalization(X0)

  X1 = trans_data

  if sample_size is not None:
      X1, formed_clusters = check_X_y(X1, formed_clusters, accept_sparse=["csc", "csr"])
      random_state = check_random_state(random_state)
      indices = random_state.permutation(X1.shape[0])[:sample_size]
      if metric == "precomputed":
          X1, formed_clusters = X1[indices].T[indices].T, formed_clusters[indices]
      else:
          X1, formed_clusters = X1[indices], formed_clusters[indices]
  return np.mean(silhouette_samples(X1, formed_clusters, metric=metric, **kwds))

#return silhouette_score1(input, formed_clusters)





In [21]:
st_rmse(data, classification)

,0,1,2,3,4,5,6
0,16.29,32.61,34.23,39.5,33.99,30.8,43.64
1,32.61,15.4,22.39,37.59,27.49,23.06,22.49
2,34.23,22.39,15.75,43.3,28.96,30.7,28.79
3,39.5,37.59,43.3,14.39,28.56,25.55,34.05
4,33.99,27.49,28.96,28.56,16.39,31.29,23.22
5,30.8,23.06,30.7,25.55,31.29,12.28,32.22
6,43.64,22.49,28.79,34.05,23.22,32.22,13.21


In [24]:
st_spat_corr(data, classification)

,0,1,2,3,4,5,6
0,0.98,0.91245,0.967213,0.973656,0.957208,0.921937,0.960006
1,0.91245,0.96,0.90792,0.937644,0.919852,0.974545,0.947056
2,0.967213,0.90792,0.97,0.94695,0.942402,0.948331,0.968044
3,0.973656,0.937644,0.94695,0.93,0.991281,0.933318,0.977881
4,0.957208,0.919852,0.942402,0.991281,0.9,0.920902,0.976925
5,0.921937,0.974545,0.948331,0.933318,0.920902,0.99,0.965089
6,0.960006,0.947056,0.968044,0.977881,0.976925,0.965089,0.95


In [25]:
st_davies(data, classification)

,0,1,2,3,4,5,6
0,16.29,32.61,34.23,39.5,33.99,30.8,43.64
1,32.61,15.4,22.39,37.59,27.49,23.06,22.49
2,34.23,22.39,15.75,43.3,28.96,30.7,28.79
3,39.5,37.59,43.3,14.39,28.56,25.55,34.05
4,33.99,27.49,28.96,28.56,16.39,31.29,23.22
5,30.8,23.06,30.7,25.55,31.29,12.28,32.22
6,43.64,22.49,28.79,34.05,23.22,32.22,13.21


In [26]:
silhouette_score1(data, classification)

0.2627598880220075

#External Cluster Evaluation

Gini Index

Entropy